# 08 word imbedding

In [6]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import fetch_20newsgroups
from tensorflow.keras.preprocessing.text import Tokenizer
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print('총 샘플 수 :',len(documents))
news_df = pd.DataFrame({'document':documents})
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())
news_df.isnull().values.any()
news_df.replace("", float("NaN"), inplace=True)
news_df.isnull().values.any()
news_df.dropna(inplace=True)
print('총 샘플 수 :',len(news_df))

총 샘플 수 : 11314
총 샘플 수 : 11004


In [13]:
nltk.download('stopwords')
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
tokenized_doc = tokenized_doc.to_list()
drop_train = [index for index, sentence in enumerate(tokenized_doc) if len(sentence) <= 1]
print(f"{len(tokenized_doc)}")
print(len(drop_train))
tokenized_doc = [sentence for index, sentence in enumerate(tokenized_doc) if index not in drop_train]
print('총 샘플 수 :',len(tokenized_doc))
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(tokenized_doc)
word2idx = tokenizer.word_index
idx2word = {value : key for key, value in word2idx.items()}
encoded = tokenizer.texts_to_sequences(tokenized_doc)
print(encoded[:2])
vocab_size = len(word2idx) + 1
print('단어 집합의 크기 :', vocab_size)
 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/choijunhyuk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


11004
43
총 샘플 수 : 10961
[[40, 53, 927, 143, 15889, 1684, 546, 279, 871, 12028, 17773, 24007, 29726, 279, 871, 63435, 871, 1128, 1103, 1998, 851, 29727, 913, 731, 20477, 279, 871, 170, 143, 1811, 149, 279, 20478, 17773, 6645, 5710, 76, 63436, 7, 36, 165, 614, 653, 29728, 6911, 24008, 2082, 829, 17774, 1119, 8790, 355, 1072, 15890, 671, 57, 163, 4231, 7206, 1933, 440, 56, 282, 4730, 9275, 2690, 39306], [1283, 429, 3, 52, 6164, 159, 112, 474, 89, 17775, 18, 63, 4731, 2865, 63437, 1042, 402, 39307, 8791, 902, 44, 8328, 316, 13041, 902, 3452, 5923, 533, 18, 87, 4732, 9872, 160, 1403, 120, 151, 5194, 63438, 63439, 17776, 63440, 13041, 903, 63441, 63442, 11172, 17777]]
단어 집합의 크기 : 181839


In [17]:
import gensim
import urllib.request

word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)   

------------------

In [20]:
from tensorflow.keras.preprocessing.sequence import skipgrams

skip_grams = [skipgrams(sample, vocabulary_size=vocab_size, window_size=10) for sample in encoded[:10]]

In [21]:
# pairs, labels = skip_grams[0][0], skip_grams[0][1]
# for i in range(5):
#     print("({:s} ({:d}), {:s} ({:d})".format(
#         idx2word[pairs[i][0]]
#     ))

In [23]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/choijunhyuk/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [24]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

In [26]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x153e3cdc0>)

In [28]:
targetXML = open('ted_en-20160408.xml', 'r', encoding="UTF8")
target_text = etree.parse(targetXML)
parse_text = "\n".join(target_text.xpath("//content/text()"))
content_text = re.sub(r'\([^)]*\)', '', parse_text)
sent_text = sent_tokenize(content_text)
normalized_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)

res = [word_tokenize(sentence) for sentence in normalized_text]

In [29]:
print("Total sample numbers : {}".format(len(res)))

Total sample numbers : 273424


In [30]:
from gensim.models import Word2Vec, FastText

# model = Word2Vec(sentences=res, vector_size=100, })

In [31]:
model = FastText(res, vector_size=100, window=5, min_count=5, workers=4, sg=1)

In [32]:
model.wv.most_similar("electrofishing")

[('electrolux', 0.8675845265388489),
 ('electrolyte', 0.8672700524330139),
 ('electroshock', 0.854694128036499),
 ('electro', 0.8530812859535217),
 ('electroencephalogram', 0.8423311114311218),
 ('electrogram', 0.8274851441383362),
 ('electrochemical', 0.8241797685623169),
 ('electric', 0.8189011812210083),
 ('electrode', 0.8187267780303955),
 ('electromagnet', 0.8159869909286499)]

In [50]:
import re
import pandas as pd
import urllib.request
from tqdm import tqdm
import hgtk
from konlpy.tag import Mecab



In [52]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")

total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
print('전체 리뷰 개수 :',len(total_data))

전체 리뷰 개수 : 200000


In [53]:
hgtk.checker.is_hangul('ㄱ')
hgtk.checker.is_hangul('28')

False

In [44]:
hgtk.letter.compose('ㄴ', 'ㅏ', 'ㅁ')
# hgtk.letter.compose('ㄴ', 'ㅏ', 'ㅁ', 'ㅁ') // parameter numbers error
# hgtk.letter.compose('ㄴ', 'ㅏ', 'ㅏ') //invalid

'남'

In [54]:
def word_to_jamo(token):
    def to_special_token(jamo):
        if not jamo: return '-'
        else:
            return jamo
    decomposed_token = ''
    for char in token:
        try:
            cho, jung, jong = hgtk.letter.decompose(char)
            cho = to_special_token(cho)
            jung = to_special_token(jung)
            jong = to_special_token(jong)
            decomposed_token = decomposed_token + cho + jung + jong
        except Exception as exception:
            if type(exception).__name__ == 'NotHangulException':
                decomposed_token += char
    return decomposed_token

word_to_jamo('남동생')
word_to_jamo('여동생')
 

'ㅇㅕ-ㄷㅗㅇㅅㅐㅇ'

In [59]:
!pip install mecab_python-0.996_ko_0.9.2_msvc-cp37-cp37m-win_amd64.whl

mecab = Mecab(r'C:\mecab\mecab-ko-dic')
print(mecab.morphs('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))
def tokenize_by_jamo(s):
    return [word_to_jamo(token) for token in mecab.morphs(s)]
print(tokenize_by_jamo('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))
tokenized_data = []
for sample in tqdm(total_data['reviews'].to_list()):
    tokenzied_sample = tokenize_by_jamo(sample)
    tokenized_data.append(tokenzied_sample)
len(tokenized_data)
tokenized_data[0]

ERROR: mecab_python-0.996_ko_0.9.2_msvc-cp37-cp37m-win_amd64.whl is not a supported wheel on this platform.


Exception: Install MeCab in order to use it: http://konlpy.org/en/latest/install/

In [58]:
def jamo_to_word(jamo_sequence):
  tokenized_jamo = []
  index = 0
  while index < len(jamo_sequence):
    if not hgtk.checker.is_hangul(jamo_sequence[index]):
      tokenized_jamo.append(jamo_sequence[index])
      index = index + 1
    else:
      tokenized_jamo.append(jamo_sequence[index:index + 3])
      index = index + 3
  word = ''
  try:
    for jamo in tokenized_jamo:
      if len(jamo) == 3:
        if jamo[2] == "-":
          word = word + hgtk.letter.compose(jamo[0], jamo[1])
        else:
          word = word + hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
      else:
        word = word + jamo
  except Exception as exception:
    if type(exception).__name__ == 'NotHangulException':
      return jamo_sequence
  return word

jamo_to_word(word_to_jamo('남동생'))

'남동생'

In [ ]:
toknized_data = []
